# This notebook is used to visualise the processed events.

In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 8})
sys.path.append(os.path.dirname(os.path.abspath('./')))
from src_visualisation.visual_utils.utils_dynamic import *
from src_visualisation.visual_utils.utils_tabfig import reconstruction_error

path_raw = '../RawData/SHRP2/'
path_processed = '../ProcessedData/SHRP2/'

## Count the number of events

In [3]:
category = {'SafeBaseline': 'Safe interactions',
            'NearCrash': 'Near-crashes',
            'Crash': 'Crashes',
            'NearCrash-NearCrash': 'Near-crashes',
            'SecondaryNearCrash': 'Near-crashes',
            'NearCrash-CrashRelevant': 'Near-crashes',
            'NearCrash-Crash': 'Crashes',
            'Crash-Crash': 'Crashes',
            'Crash-NearCrash': 'Crashes',
            'CrashRelevant-NearCrash': 'Near-crashes',
            'SecondaryCrash': 'Crashes',
            'NearCrash-OtherConflict': 'Near-crashes',
            'Crash-OtherConflict': 'Crashes',
            'CrashRelevant-Crash': 'Crashes',
            'Crash-CrashRelevant': 'Crashes'}

In [ ]:
meta_both = pd.read_csv(path_processed + 'metadata_birdseye.csv').set_index('event_id')
events = pd.read_csv(path_raw + 'FileToUse/InsightTables/Event_Table.csv').set_index('eventID')
events = events[['eventStart', 'eventEnd', 'impactProximity']]
events = events.loc[meta_both[meta_both['event_category']!='SafeBaseline'].index.values].astype(int)
events.loc[events['eventEnd']<0, 'eventEnd'] = np.nan
events.loc[events['impactProximity']<0, 'impactProximity'] = np.nan

event_counts = []
event_counts.append(meta_both['event_category'].value_counts().rename('Originally recorded'))

meta_both = meta_both[meta_both['ego_reconstructed'].astype(bool)]
event_counts.append(meta_both['event_category'].value_counts().rename('Subject reconstructed'))

meta_both = meta_both[meta_both['surrounding_reconstructed'].astype(bool)]
event_counts.append(meta_both['event_category'].value_counts().rename('Both subject and object reconstructed'))

event_counts = pd.concat(event_counts, axis=1).fillna(0).astype(int)
event_counts = event_counts.reset_index()
event_counts['Event category'] = [category[i] for i in event_counts['event_category'].values]
event_counts = event_counts.rename(columns={'event_category': 'Sub category'})
event_counts = event_counts.sort_values(by=['Event category', 'Originally recorded'], ascending=[False, False]).set_index('Event category')
event_counts.loc['Events in total (excluding SafeBaseline)'] = event_counts.iloc[1:].sum()
event_counts.loc['Crashes in total'] = event_counts.loc[event_counts.index.str.contains('Crashes')].sum()
event_counts.loc['Near-crashes in total'] = event_counts.loc[event_counts.index.str.contains('Near-crashes')].sum()
event_counts.loc[['Crashes in total', 'Near-crashes in total'], 'Sub category'] = ['', '']
event_counts.to_csv(path_processed + 'event_counts.csv')

event_categories = meta_both['event_category'].value_counts().index.values[::-1]
event_counts

C:\Users\yirujiao\AppData\Local\Temp\ipykernel_20432\1250361022.py:2: DtypeWarning: Columns (4,5,6,10,11,12,47,54) have mixed types. Specify dtype option on import or set low_memory=False.
  events = pd.read_csv(path_raw + 'FileToUse/InsightTables/Event_Table.csv').set_index('eventID')


,Sub category,Originally recorded,Subject reconstructed,Both subject and object reconstructed
Event category,,,,
Safe interactions,SafeBaseline,32509,10919,3893
Near-crashes,NearCrash,6160,5582,4648
Near-crashes,NearCrash-NearCrash,512,463,391
Near-crashes,SecondaryNearCrash,123,111,99
Near-crashes,NearCrash-CrashRelevant,112,106,88
Near-crashes,CrashRelevant-NearCrash,31,28,25
Near-crashes,NearCrash-OtherConflict,15,14,11
Crashes,Crash,1726,1607,1242
Crashes,NearCrash-Crash,79,74,61


In [ ]:
for event_type in event_categories:
    if event_type == 'SafeBaseline':
        continue
    data_ego = pd.read_hdf(path_processed + event_type + '/Ego_birdseye.h5', key='data')
    data_sur = pd.read_hdf(path_processed + event_type + '/Surrounding_birdseye.h5', key='data')
    initial_time = data_ego.groupby('event_id')['time'].min() * 1000
    events.loc[initial_time.index.values, 'recordStart'] = initial_time.values.astype(int)

events['period before impact'] = (events['impactProximity']-events['recordStart'])/1000
tta = events.loc[meta_both[(meta_both['event_category']!= 'SafeBaseline')&
                           (meta_both['ego_reconstructed'].astype(bool))&
                           (meta_both['surrounding_reconstructed'].astype(bool))].index.values, 'period before impact'].dropna()
print('Mean period before impact: ', round(tta.mean(),2), 's')

In [ ]:
event_type = event_categories[4]
data_ego = pd.read_hdf(path_processed + event_type + '/Ego_birdseye.h5', key='data')
data_sur = pd.read_hdf(path_processed + event_type + '/Surrounding_birdseye.h5', key='data')

fig = reconstruction_error(data_ego, data_sur, event_type)

## Visualize BEV trajectories

In [ ]:
event_list = data_sur[data_sur['v_ekf']<0.1]['event_id'].unique()

In [ ]:
i = -1

In [ ]:
i += 1
event_id = event_list[i] #data_ego['event_id'].unique()[i]

df_ego = data_ego[data_ego['event_id'] == event_id]
df_sur = data_sur[data_sur['event_id'] == event_id]

print(df_sur['target_id'].nunique())
if len(df_sur)==0:
    print('No surrounding data collected for trip ', event_id)
else:
    visualize_trip(df_ego, df_sur, event_id, ekf=True) # ekf=True for EKF data, False for raw data